## Assignment 1 - COMP 8730
#### Reza Barzegar - 110115742
#### Hamed Waezi - 11011****
##### 29-1-2023

#### Importing Libraries

In [1]:
import json
import time
from multiprocessing import Pool
from utils.utils import *
import nltk
from nltk.corpus import wordnet
import pytrec_eval

#### downloading wordnet and loading Birkbeck's misspelled words

In [2]:
wordnet_ = nltk.download("wordnet")
if not wordnet_:
    print("unable to download Wordnet")
dictionary = list(wordnet.words(lang='eng'))
tokens_pairs = read_data("data/missp.dat")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Reza\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Running without parallelism for 6 misspelled words

In [7]:
start = time.time()
results = []
for item in tokens_pairs[:6]:
    results.append(most_similar_words(item, dictionary))

print("/\-\/"*10)
print(f'runtime: {time.time() - start}')
print("/\-\/"*10)

/\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\/
runtime: 29.474133014678955
/\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\/


#### Running with parallelism for 6 instances

In [8]:
start = time.time()
with Pool(6) as pool:
    args = list(zip(tokens_pairs[:6], [dictionary for i in range(len(tokens_pairs[:6]))]))
    results = pool.starmap(most_similar_words, args)
print("/\-\/"*10)
print(f'runtime: {time.time() - start}')
print("/\-\/"*10)

/\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\/
runtime: 8.548056602478027
/\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\/


#### Main run with Evaluation

In [6]:
start = time.time()
with Pool(6) as pool:
    args = list(zip(tokens_pairs, [dictionary for i in range(len(tokens_pairs))]))
    results = pool.starmap(most_similar_words, args)
print("/\-\/"*10)
print(f'runtime: {time.time() - start}')
print("/\-\/"*10)
qrel = {}
for i in range(len(tokens_pairs)):
    qrel[f'{tokens_pairs[i]}'] = {
        'success_at_1': 1,
        'success_at_5': 1,
        'success_at_10': 1
    }
result_data = {}
for i in range(len(results)):
    success_at_1, success_at_5, success_at_10 = success_at_k(results[i], tokens_pairs[i])
    result_data[f'{tokens_pairs[i]}'] = {
        'success_at_1': success_at_1,
        'success_at_5': success_at_5,
        'success_at_10': success_at_10
    }

print(json.dumps(result_data, indent=1))
for measure in sorted(list(result_data[list(result_data.keys())[0]].keys())):
    print(measure, 'average:', round(pytrec_eval.compute_aggregated_measure(
              measure, [query_measures[measure] for query_measures in result_data.values()]), 2))

/\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\/
runtime: 51581.926453351974
/\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\//\-\/


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
for measure in sorted(list(result_data[list(result_data.keys())[0]].keys())):
    print(measure, 'average:',pytrec_eval.compute_aggregated_measure(
              measure, [query_measures[measure] for query_measures in result_data.values()]))

success_at_1 average: 0.27862489603548657
success_at_10 average: 0.502411976711949
success_at_5 average: 0.4453839756029942
